### Load Functions

In [1]:
from simple_trade import download_data, compute_indicator
from simple_trade import CrossTradeBacktester
from simple_trade import Optimizer

### Load Data

In [2]:
ticker = "AAPL"
start_date = "2020-01-01"
end_date = "2023-12-31"

data = download_data(ticker, start_date, end_date)

### Load Optimization Parameters

In [3]:
# --- Optimization Parameters ---
# Define the parameter grid to search
param_grid = {
    'short_window': [10, 20, 30],
    'long_window': [50, 100, 150],
}

# Define constant parameters for the backtester
initial_capital = 100000
commission_fee_long = 0.001 # 0.1%
commission_fee_short = 0.001 # 0.1%
constant_params = {
    'initial_cash': initial_capital, 
    'commission_long': commission_fee_long,
    'commission_short': commission_fee_short,
    'price_col': 'Close'
}

# Define the metric to optimize and whether to maximize or minimize
metric_to_optimize = 'total_return_pct'
maximize_metric = True

### Define Wrapper Function

In [4]:
# Define a wrapper function to handle computing indicators and running the backtest
def run_cross_trade_with_windows(data, short_window, long_window, **kwargs):
    # Work on a copy of the data
    df = data.copy()
    
    # Compute the SMA indicators
    df = compute_indicator(df, indicator='sma', parameters={'window': short_window}, columns={'close_col': 'Close'})
    df = compute_indicator(df, indicator='sma', parameters={'window': long_window}, columns={'close_col': 'Close'})
    
    # Get the indicator column names
    short_window_indicator = f"SMA_{short_window}"
    long_window_indicator = f"SMA_{long_window}"
    
    # Create a backtester instance
    backtester = CrossTradeBacktester(
        initial_cash=kwargs.pop('initial_cash', 10000),
        commission_long=kwargs.pop('commission_long', 0.001),
        commission_short=kwargs.pop('commission_short', 0.001),
        short_borrow_fee_inc_rate=kwargs.pop('short_borrow_fee_inc_rate', 0.0),
        long_borrow_fee_inc_rate=kwargs.pop('long_borrow_fee_inc_rate', 0.0)
    )
    
    # Run the backtest
    return backtester.run_cross_trade(
        data=df,
        short_window_indicator=short_window_indicator,
        long_window_indicator=long_window_indicator,
        **kwargs
    )

### Start Optimizer

In [5]:
# --- Instantiate and Run Optimizer ---
print("Initializing Optimizer...")
optimizer = Optimizer(
    data=data,
    backtest_func=run_cross_trade_with_windows,  # Use our wrapper function
    param_grid=param_grid,
    metric_to_optimize=metric_to_optimize,
    maximize_metric=maximize_metric,
    constant_params=constant_params
)

print("\nRunning Optimization (Parallel)...")
# Run optimization with parallel processing (adjust n_jobs as needed)
results = optimizer.optimize(parallel=True, n_jobs=-1) # n_jobs=-1 uses all available cores

# --- Display Results ---
print("\n--- Optimization Results ---")

# Unpack results
best_params, best_metric_value, all_results = results

print("\n--- Top 5 Parameter Combinations ---")
# Sort results for display
sorted_results = sorted(all_results, key=lambda x: x[1], reverse=maximize_metric)
for i, (params, metric_val) in enumerate(sorted_results[:5]):
    print(f"{i+1}. Params: {params}, Metric: {metric_val:.4f}")

Initializing Optimizer...
Generated 9 parameter combinations.

Running Optimization (Parallel)...
Starting optimization for 9 combinations...
Metric: total_return_pct (Maximize) | Parallel: True (n_jobs=-1)
Using 16 parallel jobs.


[Parallel(n_jobs=16)]: Using backend LokyBackend with 16 concurrent workers.
[Parallel(n_jobs=16)]: Done   2 out of   9 | elapsed:    9.9s remaining:   35.0s
[Parallel(n_jobs=16)]: Done   4 out of   9 | elapsed:   10.1s remaining:   12.7s
[Parallel(n_jobs=16)]: Done   6 out of   9 | elapsed:   10.3s remaining:    5.1s


Optimization finished in 10.69 seconds.
Best Parameters found: {'short_window': 10, 'long_window': 50}
Best Metric Value (total_return_pct): 89.0500

--- Optimization Results ---

--- Top 5 Parameter Combinations ---
1. Params: {'short_window': 10, 'long_window': 50}, Metric: 89.0500
2. Params: {'short_window': 20, 'long_window': 50}, Metric: 76.6100
3. Params: {'short_window': 30, 'long_window': 50}, Metric: 60.6400
4. Params: {'short_window': 10, 'long_window': 150}, Metric: 19.4100
5. Params: {'short_window': 20, 'long_window': 100}, Metric: 10.9600


[Parallel(n_jobs=16)]: Done   9 out of   9 | elapsed:   10.6s finished
